In [8]:
# Cell 1: Imports & config
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import joblib

# sklearn metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# LightGBM
import lightgbm as lgb

# Repro
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# ---- User config: chỉnh đường dẫn nếu cần ----
CSV_PATH = r"D:\vhproj\power-saving\data\kpi_15_mins.csv"  # <--- đổi nếu cần
OUT_DIR = "lightgbm_daily_out"
LOOKBACK_DAYS = 7        # 1 week history
TARGET_COL = "daily_ps_traffic"  # target (daily sum of ps_traffic_mb)
ACCURACY_THRESHOLD = 0.10  # ±10%
MAX_ROUNDS = 2000
EARLY_STOP = 200

os.makedirs(OUT_DIR, exist_ok=True)
print("Config ready. CSV_PATH =", CSV_PATH)


Config ready. CSV_PATH = D:\vhproj\power-saving\data\kpi_15_mins.csv


In [9]:
# Cell 2: load CSV + parse date_hour -> datetime
def robust_parse_date_hour(s):
    try:
        return pd.to_datetime(s, format='%Y-%m-%d-%H')
    except Exception:
        try:
            # fallback general parse
            return pd.to_datetime(s, errors='coerce')
        except Exception:
            return pd.NaT

df = pd.read_csv(CSV_PATH)
df.columns = [c.strip() for c in df.columns]
if 'date_hour' not in df.columns:
    raise ValueError("CSV phải có cột 'date_hour' (ví dụ '2025-10-11-00')")

# Parse to datetime and also create date (day-level)
df['date_hour_dt'] = df['date_hour'].astype(str).apply(robust_parse_date_hour)
n_before = len(df)
df = df.dropna(subset=['date_hour_dt'])
print(f"Dropped {n_before - len(df)} rows due to unparseable date_hour (if any).")
df['date'] = df['date_hour_dt'].dt.date  # python date type for grouping by day

# Ensure KPI numeric
kpi_cols = ['ps_traffic_mb', 'avg_rrc_connected_user', 'prb_dl_used', 'prb_dl_available_total']
for c in kpi_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0.0)
    else:
        print(f"Warning: {c} not found in CSV - creating zeros.")
        df[c] = 0.0

# ensure cell_name/enodeb present
if 'cell_name' not in df.columns:
    raise ValueError("CSV phải có cột 'cell_name'")
if 'enodeb' not in df.columns:
    df['enodeb'] = 'ENODEB_UNKNOWN'

print("Loaded rows:", len(df))
df.head()


Dropped 0 rows due to unparseable date_hour (if any).
Loaded rows: 313186


,update_time,enodeb,cell_name,ps_traffic_mb,avg_rrc_connected_user,prb_dl_used,prb_dl_available_total,date_hour,date_hour_dt,date
0,00:00.0,EnodebA,EnodebA3,161.14,8.527778,67.5,15.0,2025-10-11-00,2025-10-11,2025-10-11
1,00:00.0,EnodebA,EnodebA1111B031,733.10,22.038889,377.5,15.0,2025-10-11-00,2025-10-11,2025-10-11
2,00:00.0,EnodebA,EnodebA61B281,141.40,7.872223,127.5,15.0,2025-10-11-00,2025-10-11,2025-10-11
3,00:00.0,EnodebE,EnodebE61B282,0.00,0.000000,0.0,0.0,2025-10-11-00,2025-10-11,2025-10-11
4,00:00.0,EnodebD,EnodebD51B281,899.32,5.766667,638.0,15.0,2025-10-11-00,2025-10-11,2025-10-11


In [10]:
# Cell 3: aggregate 15-min -> daily per cell
daily = df.groupby(['cell_name', 'enodeb', 'date']).agg({
    'ps_traffic_mb': 'sum',               # target (daily sum)
    'avg_rrc_connected_user': 'mean',
    'prb_dl_used': 'mean',
    'prb_dl_available_total': 'mean'
}).reset_index()

# rename target column
daily = daily.rename(columns={'ps_traffic_mb': 'daily_ps_traffic'})
# ensure date is datetime.date; convert to datetime for sorting convenience
daily['date'] = pd.to_datetime(daily['date'])
daily = daily.sort_values(['cell_name', 'date']).reset_index(drop=True)

print("Daily rows (cell-day):", len(daily))
daily.head()


Daily rows (cell-day): 3348


,cell_name,enodeb,date,daily_ps_traffic,avg_rrc_connected_user,prb_dl_used,prb_dl_available_total
0,EnodebA1,EnodebA,2025-10-11,79020.73,20.174317,287.670213,14.531915
1,EnodebA1,EnodebA,2025-10-12,67587.17,16.820351,221.078947,15.000000
2,EnodebA1,EnodebA,2025-10-13,52523.90,13.661112,187.088542,15.000000
3,EnodebA1,EnodebA,2025-10-14,64094.47,14.706482,216.692708,14.843750
4,EnodebA1,EnodebA,2025-10-15,52627.37,15.664237,163.645833,15.000000


In [11]:
# Cell 4: create lag features and target (next day)
df_daily = daily.copy()

# define numeric bases we want lags for
num_bases = [
    'daily_ps_traffic',
    'avg_rrc_connected_user',
    'prb_dl_used',
    'prb_dl_available_total'
]

# create lags 1..LOOKBACK_DAYS
for lag in range(1, LOOKBACK_DAYS + 1):
    for col in num_bases:
        df_daily[f'{col}_lag{lag}'] = df_daily.groupby('cell_name')[col].shift(lag)

# rolling mean/std for last LOOKBACK_DAYS (on daily_ps_traffic)
df_daily['daily_ps_traffic_roll_mean_7'] = df_daily.groupby('cell_name')['daily_ps_traffic'].transform(
    lambda x: x.shift(1).rolling(window=LOOKBACK_DAYS, min_periods=LOOKBACK_DAYS).mean()
)
df_daily['daily_ps_traffic_roll_std_7'] = df_daily.groupby('cell_name')['daily_ps_traffic'].transform(
    lambda x: x.shift(1).rolling(window=LOOKBACK_DAYS, min_periods=LOOKBACK_DAYS).std().fillna(0)
)

# simple trend: (lag1 - lag7) / 7
if LOOKBACK_DAYS >= 2:
    df_daily[f'daily_ps_traffic_trend_{LOOKBACK_DAYS}d'] = (
        df_daily[f'daily_ps_traffic_lag1'] - df_daily[f'daily_ps_traffic_lag{LOOKBACK_DAYS}']
    ) / float(LOOKBACK_DAYS)

# target: next day's daily_ps_traffic
df_daily['target_nextday_daily_ps_traffic'] = df_daily.groupby('cell_name')['daily_ps_traffic'].shift(-1)

# drop rows missing required lag history or target
required_lags = [f'daily_ps_traffic_lag{l}' for l in range(1, LOOKBACK_DAYS + 1)]
required = required_lags + ['target_nextday_daily_ps_traffic']
df_daily = df_daily.dropna(subset=required).reset_index(drop=True)

print("Rows after creating lags & target:", len(df_daily))
df_daily.head()


Rows after creating lags & target: 2484


,cell_name,enodeb,date,daily_ps_traffic,avg_rrc_connected_user,prb_dl_used,prb_dl_available_total,daily_ps_traffic_lag1,avg_rrc_connected_user_lag1,prb_dl_used_lag1,...,prb_dl_used_lag6,prb_dl_available_total_lag6,daily_ps_traffic_lag7,avg_rrc_connected_user_lag7,prb_dl_used_lag7,prb_dl_available_total_lag7,daily_ps_traffic_roll_mean_7,daily_ps_traffic_roll_std_7,daily_ps_traffic_trend_7d,target_nextday_daily_ps_traffic
0,EnodebA1,EnodebA,2025-10-18,59425.53,14.519123,202.131579,15.0,42465.86,13.753648,153.638298,...,221.078947,15.00000,79020.73,20.174317,287.670213,14.531915,57867.364286,12893.687385,-5222.124286,67849.11
1,EnodebA1,EnodebA,2025-10-19,67849.11,16.435880,246.781250,15.0,59425.53,14.519123,202.131579,...,187.088542,15.00000,67587.17,16.820351,221.078947,15.000000,55068.050000,9106.709270,-1165.948571,72796.03
2,EnodebA1,EnodebA,2025-10-20,72796.03,19.169264,242.065217,15.0,67849.11,16.435880,246.781250,...,216.692708,14.84375,52523.90,13.661112,187.088542,15.000000,55105.470000,9167.062868,2189.315714,78243.40
3,EnodebA1,EnodebA,2025-10-21,78243.40,22.390410,229.194737,15.0,72796.03,19.169264,242.065217,...,163.645833,15.00000,64094.47,14.706482,216.692708,14.843750,58001.488571,11193.697589,1243.080000,51306.94
4,EnodebA1,EnodebA,2025-10-22,51306.94,18.850180,192.983871,15.0,78243.40,22.390410,229.194737,...,157.187500,15.00000,52627.37,15.664237,163.645833,15.000000,60022.764286,13514.220049,3659.432857,53487.46


In [12]:
# Cell 5: one-hot enodeb + select features
df_enc = pd.get_dummies(df_daily, columns=['enodeb'], drop_first=True)

# select features: all lags of daily_ps_traffic + other lags + rolling + trend + enodeb dummies
feat_cols = []

# daily_ps_traffic lags
feat_cols += [f'daily_ps_traffic_lag{l}' for l in range(1, LOOKBACK_DAYS + 1)]
# other KPI lags (means)
feat_cols += [f'avg_rrc_connected_user_lag{l}' for l in range(1, LOOKBACK_DAYS + 1)]
feat_cols += [f'prb_dl_used_lag{l}' for l in range(1, LOOKBACK_DAYS + 1)]
feat_cols += [f'prb_dl_available_total_lag{l}' for l in range(1, LOOKBACK_DAYS + 1)]
# rolling + trend
feat_cols += ['daily_ps_traffic_roll_mean_7', 'daily_ps_traffic_roll_std_7', f'daily_ps_traffic_trend_{LOOKBACK_DAYS}d']

# add enodeb dummies
enodeb_cols = [c for c in df_enc.columns if c.startswith('enodeb_')]
feat_cols += enodeb_cols

# ensure feature columns exist
feat_cols = [c for c in feat_cols if c in df_enc.columns]

X = df_enc[feat_cols].fillna(0)
y = df_enc['target_nextday_daily_ps_traffic'].astype(float)
meta = df_enc[['cell_name', 'date', 'daily_ps_traffic', 'target_nextday_daily_ps_traffic']].copy()

print("Num features:", len(feat_cols))
X.head()


Num features: 40


,daily_ps_traffic_lag1,daily_ps_traffic_lag2,daily_ps_traffic_lag3,daily_ps_traffic_lag4,daily_ps_traffic_lag5,daily_ps_traffic_lag6,daily_ps_traffic_lag7,avg_rrc_connected_user_lag1,avg_rrc_connected_user_lag2,avg_rrc_connected_user_lag3,...,daily_ps_traffic_trend_7d,enodeb_EnodebB,enodeb_EnodebC,enodeb_EnodebD,enodeb_EnodebE,enodeb_EnodebF,enodeb_EnodebG,enodeb_EnodebH,enodeb_EnodebI,enodeb_EnodebK
0,42465.86,46752.05,52627.37,64094.47,52523.90,67587.17,79020.73,13.753648,14.815336,15.664237,...,-5222.124286,False,False,False,False,False,False,False,False,False
1,59425.53,42465.86,46752.05,52627.37,64094.47,52523.90,67587.17,14.519123,13.753648,14.815336,...,-1165.948571,False,False,False,False,False,False,False,False,False
2,67849.11,59425.53,42465.86,46752.05,52627.37,64094.47,52523.90,16.435880,14.519123,13.753648,...,2189.315714,False,False,False,False,False,False,False,False,False
3,72796.03,67849.11,59425.53,42465.86,46752.05,52627.37,64094.47,19.169264,16.435880,14.519123,...,1243.080000,False,False,False,False,False,False,False,False,False
4,78243.40,72796.03,67849.11,59425.53,42465.86,46752.05,52627.37,22.390410,19.169264,16.435880,...,3659.432857,False,False,False,False,False,False,False,False,False


In [13]:
# Cell 6: time-based split by date (70/20/10)
unique_dates = sorted(df_enc['date'].dt.normalize().unique())  # normalized datetimes
n_dates = len(unique_dates)
print("Unique dates (days) available:", n_dates)

# compute counts
n_train = max(1, int(n_dates * 0.70))
n_val   = max(1, int(n_dates * 0.20))
n_test  = n_dates - n_train - n_val
if n_test < 1:
    n_test = 1
    if n_train + n_val + n_test > n_dates:
        n_val = max(0, n_val - 1)

train_dates = unique_dates[:n_train]
val_dates   = unique_dates[n_train:n_train + n_val]
test_dates  = unique_dates[n_train + n_val:]

print(f"Split days -> total: {n_dates}, train: {len(train_dates)}, val: {len(val_dates)}, test: {len(test_dates)}")

train_mask = df_enc['date'].isin(train_dates)
val_mask   = df_enc['date'].isin(val_dates)
test_mask  = df_enc['date'].isin(test_dates)

X_train, y_train = X[train_mask], y[train_mask]
X_val,   y_val   = X[val_mask],   y[val_mask]
X_test,  y_test  = X[test_mask],  y[test_mask]

print("Samples -> Train:", len(X_train), "Val:", len(X_val), "Test:", len(X_test))


Unique dates (days) available: 23
Split days -> total: 23, train: 16, val: 4, test: 3
Samples -> Train: 1728 Val: 432 Test: 324


In [ ]:
# Cell 7: Train LightGBM (tương thích mọi phiên bản LightGBM)

import lightgbm as lgb

RANDOM_SEED = 42
EARLY_STOP = 50
MAX_ROUNDS = 2000

train_data = lgb.Dataset(X_train, label=y_train)
val_data   = lgb.Dataset(X_val, label=y_val, reference=train_data)

params = {
    'objective': 'regression',
    'metric': ['l2', 'l1'],
    'boosting_type': 'gbdt',
    'num_leaves': 64,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'seed': RANDOM_SEED,
    'verbose': -1
}

# Callbacks cho mọi phiên bản LightGBM
callbacks = [
    lgb.early_stopping(stopping_rounds=EARLY_STOP, verbose=True),
    lgb.log_evaluation(period=100)
]

print("Training LightGBM...")
model_lgb = lgb.train(
    params=params,
    train_set=train_data,
    num_boost_round=MAX_ROUNDS,
    valid_sets=[train_data, val_data],
    valid_names=['train', 'valid'],
    callbacks=callbacks
)

print("Training Done. Best iteration:", model_lgb.best_iteration)


Training LightGBM...


TypeError: train() got an unexpected keyword argument 'evals_result'

In [ ]:
# Cell 8: plot training curves
plt.figure(figsize=(12,5))
if 'train' in evals_result and 'valid' in evals_result:
    plt.subplot(1,2,1)
    plt.plot(evals_result['train']['l2'], label='train_l2 (MSE)')
    plt.plot(evals_result['valid']['l2'], label='valid_l2 (MSE)')
    plt.xlabel('Boosting rounds'); plt.ylabel('MSE'); plt.legend(); plt.title('MSE per round')

    plt.subplot(1,2,2)
    plt.plot(evals_result['train']['l1'], label='train_l1 (MAE)')
    plt.plot(evals_result['valid']['l1'], label='valid_l1 (MAE)')
    plt.xlabel('Boosting rounds'); plt.ylabel('MAE'); plt.legend(); plt.title('MAE per round')
    plt.tight_layout()
    plt.show()
else:
    print("No evals_result to plot.")


In [ ]:
# Cell 9: predict + metrics
y_pred_test = model_lgb.predict(X_test, num_iteration=model_lgb.best_iteration)

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
with np.errstate(divide='ignore', invalid='ignore'):
    mape_vals = np.abs((y_test.values - y_pred_test) / np.where(y_test.values == 0, np.nan, y_test.values))
mape = np.nanmean(mape_vals) * 100
r2 = r2_score(y_test, y_pred_test)

# accuracy within ±threshold
abs_err = np.abs(y_test.values - y_pred_test)
rel_condition = np.where(y_test.values == 0, abs_err <= 10.0, abs_err <= (ACCURACY_THRESHOLD * np.abs(y_test.values)))
accuracy_pct = np.mean(rel_condition) * 100.0

print("=== TEST METRICS (LightGBM) ===")
print(f"Loss (MSE) : {mse:.6f}")
print(f"MAE        : {mae:.6f}")
print(f"RMSE       : {rmse:.6f}")
print(f"MAPE       : {mape:.2f} %")
print(f"R2         : {r2:.4f}")
print(f"Accuracy (within ±{int(ACCURACY_THRESHOLD*100)}%): {accuracy_pct:.2f} %")


In [ ]:
# Cell 10: save model & predictions
model_path = os.path.join(OUT_DIR, 'lightgbm_daily_model.txt')
model_lgb.save_model(model_path)
print("Saved model to:", model_path)

# save feature list
joblib.dump(feat_cols, os.path.join(OUT_DIR, 'feature_columns.joblib'))
print("Saved feature list.")

# save test predictions with meta info
test_df = df_enc[test_mask].copy().reset_index(drop=True)  # df_enc contains original meta and encoding
test_df['pred_nextday'] = y_pred_test
out_test_csv = os.path.join(OUT_DIR, 'test_predictions_daily_lightgbm.csv')
test_df[['cell_name', 'date', 'daily_ps_traffic', 'target_nextday_daily_ps_traffic', 'pred_nextday']].to_csv(out_test_csv, index=False)
print("Saved test predictions to:", out_test_csv)

# show sample
print("\nSample predictions (first 12 rows):")
display_cols = ['cell_name', 'date', 'daily_ps_traffic', 'target_nextday_daily_ps_traffic', 'pred_nextday']
print(test_df[display_cols].head(12).to_string(index=False))


In [ ]:
# Cell 11: predict next day for latest day rows (use last available day per cell)
latest_date = df_enc['date'].max()
print("Latest date in data:", latest_date)
latest_rows = df_enc[df_enc['date'] == latest_date]
if not latest_rows.empty:
    X_latest = latest_rows[feat_cols].fillna(0)
    preds_nextday = model_lgb.predict(X_latest, num_iteration=model_lgb.best_iteration)
    out_next = latest_rows[['cell_name', 'date', 'daily_ps_traffic']].copy()
    out_next['pred_next_day_daily_ps_traffic'] = preds_nextday
    out_path = os.path.join(OUT_DIR, 'pred_next_day_for_latest_date.csv')
    out_next.to_csv(out_path, index=False)
    print("Saved next-day predictions for latest date to:", out_path)
    print(out_next.head(20).to_string(index=False))
else:
    print("No rows for latest date to predict.")


In [ ]:
# Cell 12: feature importance plot
plt.figure(figsize=(8,10))
ax = lgb.plot_importance(model_lgb, max_num_features=30, height=0.4)
plt.title("LightGBM Feature Importance (top 30)")
plt.tight_layout()
fig_path = os.path.join(OUT_DIR, 'feature_importance_daily.png')
plt.savefig(fig_path, dpi=150)
print("Saved feature importance plot to:", fig_path)
plt.show()
